In [51]:
import pandas as pd
import torch

import torch.nn as nn

In [52]:
twitter_df = pd.read_csv('Data/twitter_data_clean.csv')
twitter_df['tweet'].head()

C:\Users\Zach\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


0    episode 13 crypto thunderdome part 1 prize 50 ...
1    grt chart going off the screen lambo soon btc ...
2    live bitcoin trading with deribot on deribit e...
3    deribot daily trading report 20122020 1107 utc...
4    learned about stellar amp earned xlm in return...
Name: tweet, dtype: object

In [53]:
training_df = pd.read_csv('Data/training_data_clean.csv')
training_df.sample(n=100000, random_state=100)
training_df.head()

,sentiment,tweet
0,0,awww that is a bummer you should have got davi...
1,0,is upset that he can not update his facebook b...
2,0,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no it is not behaving at all i am mad why am i...


In [54]:
training_df = training_df.dropna()

In [55]:
list_of_tweets = training_df['tweet'].tolist()
from collections import Counter

all_text = ' '.join(list_of_tweets)
words = all_text.split()

count_words = Counter(words)
total_words = len(words)
sorted_words = count_words.most_common(total_words)
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [56]:
import numpy as np

seq_length = 20

# Return features of tweet ints, where each tweet is padded with 0's or truncated to the input seq_length.
def pad_features(tweets_int, seq_length):
    
    features = np.zeros((len(tweets_int), seq_length), dtype = int)
    
    for i, tweet in enumerate(tweets_int):
        tweet_len = len(tweet)
        
        if tweet_len <= seq_length:
            zeroes = list(np.zeros(seq_length-tweet_len))
            new = zeroes+tweet
        elif tweet_len > seq_length:
            new = tweet[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features


In [57]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

In [58]:
torch.load('sentiment_analysis.pt')

C:\Users\Zach\anaconda3\lib\site-packages\torch\serialization.py:624: UserWarning: Couldn't retrieve source code for container of type SentimentLSTM. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "
C:\Users\Zach\anaconda3\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\Zach\anaconda3\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, Sourc

SentimentLSTM(
  (embedding): Embedding(67394, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [59]:
def predict(net, test_review, sequence_length=20):
    
    net.train(net, mode=False)
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [60]:
 twitter_df['sentiment'] = twitter_df.apply(lambda row: predict(SentimentLSTM, row['tweet']), axis=1)

TypeError: children() missing 1 required positional argument: 'self'